In [ ]:
import os
from datetime import datetime
from py_clob_client.client import get

from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient

host = "https://clob.polymarket.com"
key = os.getenv("POLYMARKET_PRIVATE_KEY")
chain_id = POLYGON


class MyClobClient(ClobClient):
    def get_timeseries(
        self,
        market: str,
        start_ts: datetime,
        end_ts: datetime,
        fidelity_in_minutes: int,
    ):
        """
        Parameters:
            market: aka token_id
            start_ts
            end_ts
            fidelity

        Returns:
            TODO
        """

        # Source: https://docs.polymarket.com/#timeseries-data
        request_url = "{}{}?market={}&startTs={}&endTs={}&fidelity={}".format(
            self.host,
            "/prices-history",
            market,
            int(start_ts.timestamp()),
            int(end_ts.timestamp()),
            fidelity_in_minutes,
        )
        print(request_url)
        return get(request_url)


client = MyClobClient(host, key=key, chain_id=chain_id)
client.set_api_creds(client.create_or_derive_api_creds())

In [48]:
from datetime import timedelta


ts = client.get_timeseries(
    "82927494066775506689099766141381125044040984047946118898947348013267483041764",
    datetime.now() - timedelta(days=1),
    datetime.now(),
    60,
)
ts

https://clob.polymarket.com/prices-history?market=82927494066775506689099766141381125044040984047946118898947348013267483041764&startTs=1741103758&endTs=1741190158&fidelity=60


{'history': [{'t': 1741104005, 'p': 0.245},
  {'t': 1741107605, 'p': 0.265},
  {'t': 1741111205, 'p': 0.285},
  {'t': 1741114805, 'p': 0.3},
  {'t': 1741118405, 'p': 0.33},
  {'t': 1741122005, 'p': 0.335},
  {'t': 1741125605, 'p': 0.345},
  {'t': 1741129205, 'p': 0.345},
  {'t': 1741132804, 'p': 0.345},
  {'t': 1741136404, 'p': 0.335},
  {'t': 1741140005, 'p': 0.34},
  {'t': 1741143605, 'p': 0.335},
  {'t': 1741147205, 'p': 0.33},
  {'t': 1741150805, 'p': 0.335},
  {'t': 1741154405, 'p': 0.34},
  {'t': 1741158005, 'p': 0.345},
  {'t': 1741161605, 'p': 0.345},
  {'t': 1741165205, 'p': 0.345},
  {'t': 1741168804, 'p': 0.35},
  {'t': 1741172405, 'p': 0.39},
  {'t': 1741176004, 'p': 0.39},
  {'t': 1741179604, 'p': 0.42},
  {'t': 1741183204, 'p': 0.385},
  {'t': 1741186804, 'p': 0.375},
  {'t': 1741190105, 'p': 0.365}]}

In [29]:
ss_markets = client.get_simplified_markets()
ss_markets["data"][0]

{'condition_id': '0x9deb0baac40648821f96f01339229a422e2f5c877de55dc4dbf981f95a1e709c',
 'rewards': {'rates': None, 'min_size': 0, 'max_spread': 0},
 'tokens': [{'token_id': '14270523446080509320829200481895961480205553513304203367521919818541658424782',
   'outcome': 'Chiefs',
   'price': 1,
   'winner': False},
  {'token_id': '93110170397161149119544349457822484949376809039410140245101963942162463626903',
   'outcome': 'Raiders',
   'price': 0,
   'winner': False}],
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False}

In [ ]:
import time


def find_crypto_markets(limit=10, max_queries=3, cursor="MA==", condition=lambda m: m["tags"] and "Crypto" in m["tags"]):
    found_any = False
    crypto_markets = []

    queries_made = 0
    while (queries_made < max_queries) and (len(crypto_markets) < limit):
        print(f"Fetching markets with cursor {cursor}")

        markets = client.get_markets(next_cursor=cursor)
        cursor = markets["next_cursor"]

        data = [m for m in markets["data"] if condition(m)]
        if data:
            if not found_any:
                print(f"First cursor with data found: {cursor}")
                found_any = True

        print(f"Found {len(data)} crypto markets")

        crypto_markets += data
        queries_made += 1

        time.sleep(0.5)

    markets = client.get_markets()

    if not crypto_markets:
        print("No crypto markets found")
    else:
        print(f"Crypto markets found ({len(crypto_markets)})")
    
    print(f"Next cursor: {cursor}")

    return crypto_markets


def is_btc_related_market(m):
    return m["question"] and ("btc" in m["slug"])

# def is_

markets = find_crypto_markets(limit=10, cursor="NTAwMA==", condition=is_btc_related_market)

Fetching markets with cursor NTAwMA==
['Music', 'janelle monae', 'album of the year', 'Awards', 'Taylor Swift', 'miley cyrus', 'olivia rodrigo', 'boygenius', 'world music radio', 'sza', 'lana del rey', 'jon batiste', 'entertainment', 'midnights', 'All', 'Grammys']
First cursor with data found: NTUwMA==
Found 6 crypto markets
Fetching markets with cursor NTUwMA==
['All']
Found 14 crypto markets
Crypto markets found (20)
Next cursor: NjAwMA==


In [19]:
markets[0]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': None,
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x40a93ba2e52a40676801cb144050bd4fc64ddedb778f13f2ade72d61a89dee8b',
 'question_id': '0x5e3f3582f9f098240b934c7a8143d43fdb09b13e06fce5bc85a59201f457ec62',
 'question': 'Will BTC hit $50,000 in February?',
 'description': 'This market will immediately resolve to “Yes” if any Coinbase 1 minute candle for bitcoin BTC-USD between 1 Feb \'24 00:00 and 29 Feb \'24 23:59 in the ET timezone has a final “High” price of 50,000.00 or higher. Otherwise, this market will resolve to "No".\n\nThe resolution source for this market is https://www.coinbase.com/, specifically the BTC-USD "High" prices currently available at https://www.coinbase.com/advanced-trade/BTC-USD with “1m” and “Candles” selected on the top bar.\n\nTo see the "High" prices, mouse over particular candles and look at 

In [20]:
[("https://polymarket.com/event/" + m["market_slug"], m["condition_id"]) for m in markets]


[('https://polymarket.com/event/will-btc-hit-50000-in-february',
  '0x40a93ba2e52a40676801cb144050bd4fc64ddedb778f13f2ade72d61a89dee8b'),
 ('https://polymarket.com/event/will-btc-hit-55000-in-february',
  '0x8771fe78b111e770a6d61cb3957a9667b05e06dc8fb357564be69f2d732f09fe'),
 ('https://polymarket.com/event/will-mtgox-btc-distribution-start-in-march',
  ''),
 ('https://polymarket.com/event/will-mtgox-btc-distribution-start-in-april',
  ''),
 ('https://polymarket.com/event/will-mtgox-btc-distribution-start-in-february',
  ''),
 ('https://polymarket.com/event/will-mtgox-btc-distribution-start-in-january',
  ''),
 ('https://polymarket.com/event/will-btc-hit-80000-in-march',
  '0x9680e41769b76d79cddffb9ace729d5141d8b5d94b5277461595031de5da3534'),
 ('https://polymarket.com/event/will-btc-hit-60000-in-march',
  '0x0026f721f4dd4caef4c1adff3aae56a6938141b7996a5c86e5879e80b9340e7a'),
 ('https://polymarket.com/event/archwill-btc-hit-100000-in-march',
  '0xdf37d07f56bbc6de331067f0726ca00513a4c584c

In [42]:
market = client.get_market("0xa9444a9067fb2958ac36dd3f0ea25c07da5c39dc595643280b5ec037334b75de")
market

{'enable_order_book': True,
 'active': True,
 'closed': False,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': '2025-01-07T16:59:09Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.01,
 'condition_id': '0xa9444a9067fb2958ac36dd3f0ea25c07da5c39dc595643280b5ec037334b75de',
 'question_id': '0xb6a17e6442f360824cc866b24ef43b20598fc2eebe395cebd189c48f33b5d258',
 'question': 'Bitcoin up in Q1 2025?',
 'description': 'This market will resolve to "Up" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is higher than 93,445.45.\n\nThis market will resolve to "Down" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is lower than 93,445.45.\n\nThis market will resolve to "50-50" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is of precisely 93,445.45.\n\nThe resolution source for thi

In [ ]:
token_id = "36665780724924721135862021472225286854518004551848050268584060069434509875611"

resp = client
resp

OrderBookSummary(market='0x39bc347a1bb74a3f410ea850a46962572e3d932bf00f0f99b7a3d2b4141fa87a', asset_id='36665780724924721135862021472225286854518004551848050268584060069434509875611', timestamp='1741188769836', bids=[OrderSummary(price='0.001', size='211874.23'), OrderSummary(price='0.002', size='65813.65'), OrderSummary(price='0.003', size='8633.8')], asks=[OrderSummary(price='0.999', size='3000000'), OrderSummary(price='0.99', size='128.18'), OrderSummary(price='0.97', size='333'), OrderSummary(price='0.95', size='1000'), OrderSummary(price='0.89', size='526.04'), OrderSummary(price='0.52', size='7000'), OrderSummary(price='0.519', size='15000'), OrderSummary(price='0.5', size='8.33'), OrderSummary(price='0.36', size='333.33'), OrderSummary(price='0.29', size='263.79'), OrderSummary(price='0.231', size='4000'), OrderSummary(price='0.214', size='30000'), OrderSummary(price='0.207', size='1450'), OrderSummary(price='0.19', size='210'), OrderSummary(price='0.16', size='40.07'), OrderSum

In [37]:
resp.bids[0]

OrderSummary(price='0.001', size='211874.23')